#SIS202560 - Procesamiento Inteligente de Datos

<div>

### SIS202560_Formato-Python

* 3.Redes bayesianas y cadenas de Markov
* SIS202560_Python_MirandaHau_Tarea-3


**Datos del estudiante:**
* Nombre completo: Deysi Miranda Hau Marrufo
* Mátricula: 00443416


Profesor: Enrique Naredo García

#3. Redes bayesianas y cadenas de Markov

## Introducción

- Esta práctica utiliza Redes Bayesianas para clasificar dígitos manuscritos.
- Se implementa el clasificador BNClassifier de la librería pyAgrum.
- Se realiza la división de los datos en 70% entrenamiento y 30% prueba.
- Se visualizan los resultados y se analizan las características más relevantes mediante la Markov Blanket.

In [4]:
# Instalar dependencias de pyAgrum
!apt-get install -y python3-pyqt5 python3-pyqt5.qtsvg

# Instalar pyAgrum
!pip install pyAgrum --quiet

# Instalar otras librerías necesarias
!pip install scikit-learn matplotlib numpy --quiet

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libevdev2 libgudev-1.0-0 libinput-bin libinput10 libmd4c0 libmtdev1
  libqt5core5a libqt5dbus5 libqt5designer5 libqt5gui5 libqt5help5
  libqt5network5 libqt5printsupport5 libqt5sql5 libqt5sql5-sqlite libqt5svg5
  libqt5test5 libqt5widgets5 libqt5xml5 libwacom-bin libwacom-common libwacom9
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1
  libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 libxkbcommon-x11-0
  python3-pyqt5.sip qt5-gtk-platformtheme qttranslations5-l10n
Suggested packages:
  qt5-image-formats-plugins qtwayland5
The following NEW packages will be installed:
  libevdev2 libgudev-1.0-0 libinput-bin libinput10 libmd4c0 libmtdev1
  libqt5core5a libqt5dbus5 libqt5designer5 libqt5gui5 libqt5help5
  libqt5network5 libqt5printsupport5 libqt5sql5 libqt5sql5-sqlite libqt5svg5
  libqt5test5 libqt5widgets5 libq

In [1]:
# Librerías para redes bayesianas
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
from pyAgrum.skbn import BNClassifier

# Librerías para gráficos y análisis
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import patheffects as pe

# Librerías de scikit-learn
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

ModuleNotFoundError: No module named 'pyAgrum'

##Subtitulo

Contenido y comentarios.

In [ ]:
# Comentario del código

# código probado y sin errores

##Conclusiones

Contenido y comentarios.

##Bibiliografía

[1] 1era Refeerencia en estilo APA.

[2] 2da Refeerencia en estilo APA.

[3] 3era Refeerencia en estilo APA.